In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Data Load

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
data_path = '/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2014_NSOK_preprocessed.csv'

final_data = pd.read_csv(data_path, index_col=None) ## sperater, delimiter 확인
final_data = final_data.drop('Unnamed: 0', axis=1)

In [4]:
test_2017 = pd.read_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2017_NSOK_preprocessed.csv')
test_2017 = test_2017.drop('Unnamed: 0', axis=1)

test_2020 = pd.read_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2020_NSOK_preprocessed.csv')
test_2020 = test_2020.drop('Unnamed: 0', axis=1)

In [5]:
print(final_data.shape)
print(test_2017.shape)
print(test_2020.shape)

(10281, 132)
(10083, 132)
(9920, 132)


In [6]:
## 1. one-hot encoding

# cat_list = ['A10_2_y', # 2019년 대비 가구원 변화
# 'D8_y', # 종교종류
# 'E1_2_y', # 현재 일 내용
# 'E1_3_y', # 현재 일 종사상 지위	
# 'E1_3_1_y', # 현재 일_정부지원일자리 여부
# 'E2_1_y', # 최장기 일 종사상 지위
# 'E2_3_y', # 최장기 일 그만 둔 이유
# 'E3_4_y', ## 구직활동노력
# 'F12_y', # 자녀 동거 시 일상생활 분담
# 'H10_y', # 외출 시 가장 불편한 점
# 'H22_y', # 희망 장례 방법
# 'Q1_y' # 주택종류
# ]

cat_list = ['A10_2_y',
'D8_y',
'E1_2_y',
'E1_3_y',
'E1_3_1_y',
'E2_1_y',
'E2_3_y',
'F12_y',
'Q1_y',]

final_data = pd.get_dummies(final_data, columns=cat_list)
test_2017 = pd.get_dummies(test_2017, columns=cat_list)
test_2020 = pd.get_dummies(test_2020, columns=cat_list)

In [7]:
final_data.columns = [re.sub('\.0', r'', c) for c in final_data.columns]
test_2017.columns = [re.sub('\.0', r'', c) for c in test_2017.columns]
test_2020.columns = [re.sub('\.0', r'', c) for c in test_2020.columns]

In [8]:
## leave common columns in 3 dfs
dataset = [final_data, test_2017, test_2020]
common_cols = list(set.intersection(*(set(df.columns) for df in dataset)))

final_data = final_data[common_cols]
test_2017 = test_2017[common_cols]
test_2020 = test_2020[common_cols]

print(len(common_cols))
print(len(final_data.columns))
print(len(test_2017.columns))
print(len(test_2020.columns))

179
179
179
179


In [9]:
X = final_data.drop('suicide', axis=1)
y = final_data['suicide']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=y)

In [11]:
print(len(y_train[y_train==0]), len(y_train[y_train==1]))
print(len(y_test[y_test==0]), len(y_test[y_test==1]))

6495 701
2785 300


In [12]:
test_2017_X = test_2017.drop('suicide', axis=1)
test_2017_y = test_2017['suicide']

test_2020_X = test_2020.drop('suicide', axis=1)
test_2020_y = test_2020['suicide']

In [13]:
## 2. Iterative imputation (fitting in X_train -> transform)

con_list = ['S5_1_y',
'S5_2_y',
'S6_1_y',
'cohab_mean_age',
'cohab_mean_edu_y',
'cohab_women_p',
'cohab_work_n',
'B2_3_y',
'B3_y',
'B4_1_etc_y',
'B5_1_etc_1_y',
'B8_y',
'B9_1a_y',
'bmi',
'nutrition',
'adl',
'D5_3_1_y',
'D6_1_1_y',
'D6_1_2_y',
'D6_1_3_y',
'D7_3_1_y',
'fac_use_public',
'E1_5_1_y',
'E2_2_1_y',
'num_child_out',
'meet_child_out',
'call_child_out',
'family_contact',
'friends_contact',
'num_alter',
'fac_dist',
'H12_1_etc_y',
'J1b_1_y',
'J1b_3_y',
'J1b_4_y',
'J3b_1_13_y',
'J3b_3_13_y',
'financial_asset',
'J6b_4_y',
'RES_AGE_y',
'RES_EDU2_y',
]

imputer = IterativeImputer(random_state=1120)
after_imputation = imputer.fit_transform(X_train.loc[:,con_list])

result_imputed_iter = pd.DataFrame(after_imputation)
result_imputed_iter.columns = X_train.loc[:,con_list].columns
result_imputed_iter.index = X_train.loc[:,con_list].index
X_train.loc[:,con_list] = result_imputed_iter

In [14]:
after_imputation = imputer.transform(X_test.loc[:,con_list])
result_imputed_iter = pd.DataFrame(after_imputation)
result_imputed_iter.columns = X_test.loc[:,con_list].columns
result_imputed_iter.index = X_test.loc[:,con_list].index
X_test.loc[:,con_list] = result_imputed_iter

after_imputation = imputer.transform(test_2017_X.loc[:,con_list])
result_imputed_iter = pd.DataFrame(after_imputation)
result_imputed_iter.columns = test_2017_X.loc[:,con_list].columns
result_imputed_iter.index = test_2017_X.loc[:,con_list].index
test_2017_X.loc[:,con_list] = result_imputed_iter

after_imputation = imputer.transform(test_2020_X.loc[:,con_list])
result_imputed_iter = pd.DataFrame(after_imputation)
result_imputed_iter.columns = test_2020_X.loc[:,con_list].columns
result_imputed_iter.index = test_2020_X.loc[:,con_list].index
test_2020_X.loc[:,con_list] = result_imputed_iter

In [15]:
print(X_train.isnull().sum()[X_train.isnull().sum()!=0])
print(test_2017_X.isnull().sum()[test_2017_X.isnull().sum()!=0])
print(test_2020_X.isnull().sum()[test_2020_X.isnull().sum()!=0])

Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)


In [16]:
## 3. Min-max scaling (dummy 변수를 함꼐 고려하기 때문에.)
scaler = MinMaxScaler()
X_train.loc[:,con_list] = scaler.fit_transform(X_train.loc[:,con_list])
X_test.loc[:,con_list] = scaler.fit_transform(X_test.loc[:,con_list])
test_2017_X.loc[:,con_list] = scaler.transform(test_2017_X.loc[:,con_list])
test_2020_X.loc[:,con_list] = scaler.transform(test_2020_X.loc[:,con_list])

In [17]:
print(X_train.shape)

(7196, 178)


In [18]:
## concat
train_2014_r = pd.concat([X_train, y_train],axis=1)
test_2014_r = pd.concat([X_test, y_test],axis=1)
test_2017_r = pd.concat([test_2017_X, test_2017_y],axis=1)
test_2020_r = pd.concat([test_2020_X, test_2020_y],axis=1)

In [19]:
print(train_2014_r.shape)
print(test_2014_r.shape)
print(test_2017_r.shape)
print(test_2020_r.shape)

(7196, 179)
(3085, 179)
(10083, 179)
(9920, 179)


In [20]:
train_2014_r.to_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2014_NSOK_train_preprocessed_2.csv')
test_2014_r.to_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2014_NSOK_test_preprocessed_2.csv')
test_2017_r.to_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2017_NSOK_preprocessed_2.csv')
test_2020_r.to_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2020_NSOK_preprocessed_2.csv')